In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
import onnxruntime as ort
from onnx import helper
from onnxruntime_extensions import get_library_path
import numpy as np

In [ ]:
mfile = "estimate/Mul/Mul_0_out0/partition_0.onnx"
mfile_up = mfile.replace(".onnx","_ort.onnx")

model = ModelWrapper(mfile)

def revert_quantAvgPool(model):
    nodes = [n for n in model.graph.node if n.op_type == 'QuantAvgPool2d']
    attrs = [n.attribute for n in model.graph.node if n.op_type == 'QuantAvgPool2d']
    for node,attr in zip(nodes,attrs):
        for a in attr:
            if a.name == "stride":
                s = a.i
            elif a.name == "kernel":
                k = a.i
        update = helper.make_node(
            "AveragePool",
            inputs=[node.input[0]],
            outputs=[node.output[0]],
            kernel_shape=[k,k],
            strides=[s,s],
        )

        model.graph.node.remove(node)
        model.graph.node.append(update)
        
def set_multithreshold_default(model,save_model):

    new_attr = [helper.make_attribute("out_scale", 1.0),
                helper.make_attribute("out_bias", 0.0),
                helper.make_attribute("data_layout","NCHW")]

    for n in model.graph.node:
        if n.op_type == "MultiThreshold":
            out_scale,bias,datalayout = False,False,False
            for na in n.attribute:
                if na.name == "out_scale": out_scale = True
                if na.name == "out_bias": bias = True
                if na.name == "data_layout": datlayout = True
            if not out_scale: n.attribute.append(new_attr[0])
            if not bias: n.attribute.append(new_attr[1])
            if not datalayout: n.attribute.append(new_attr[2])

            n.domain = "ai.onnx.contrib"
    model.save(mfile_up)

def ort_implementation(x,model):
    so = ort.SessionOptions()
    so.register_custom_ops_library(get_library_path())

    sess = ort.InferenceSession(mfile_up, so)
    inp_dict = {}
    for inp in sess.get_inputs():
        inp_dict[inp.name] = x
    res = sess.run([],inp_dict)
    return res   

revert_quantAvgPool(model)
set_multithreshold_default(model,mfile_up)
ort_implementation(np.load("estimate/Mul/Mul_0_out0/input.npy"),model)
